# **3: Text Extraction and Refinement** 
**3.1. Text Extraction From Govt. Documents**<br>
**3.2. Text Refinement (Data Cleansing)** <br>
>3.2.1. Level 1 Refinement<br>
>>3.2.1.1. Level 1 Refinement of OpenAI generated domain specific sentences and joining with existing sentences. <br>

>3.2.2. Level 2 Refinement<br>
>3.2.3. Level 3 Refinement<br>

`Input: Govt. Documents [PDFs] /content/drive/MyDrive/Final/PMJAY_Documents/` <br>
`Output: refined_text.csv /content/mydrive/MyDrive/Final/refine_text.csv`

##3.1. Text Extraction

In [ ]:
#Installing Tika Library for Text Extration from PDF
!pip install tika

In [2]:
import pandas as pd #For Data Manipulation
from google.colab import drive #For Getting Documents From Google Drive
drive.mount('/content/drive') #Mounting the Google Drive
from tika import parser

Mounted at /content/drive


In [3]:
# opening pdf file
pdf_1 = parser.from_file("/content/drive/MyDrive/Final (Completed)/PMJAY_Documents/Process Flow Guidebook.pdf")
pdf_2 = parser.from_file("/content/drive/MyDrive/Final (Completed)/PMJAY_Documents/PM-JAY Process Flow at Empanelled Hospitals.pdf")  
raw_text = pdf_1['content']+pdf_2['content'] 

2023-05-04 08:36:39,650 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2023-05-04 08:36:43,778 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2023-05-04 08:36:44,836 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2023-05-04 08:36:49,844 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [4]:
print(raw_text)


































Microsoft Word - Beneficiary Empowerment Guidebook copy.docx


      
   

  
  

  
 

 
 
 

AYUSHMAN BHARAT 
PRADHAN MANTRI JAN AROGYA YOJANA 

  
 
 
 

BENEFICIARY EMPOWERMENT GUIDEBOOK 
 
 

March 2019 
 
 
 
 
 
 

  



     

 2 

  



     

 3 

 

  Foreword 

 

 

 

 

 

Ayushman Bharat – Pradhan Mantri Jan Arogya Yojana (PM-JAY) was launched by Hon’ble Prime 
Minister on September 23, 2018. PM-JAY is an ambitious government scheme which intends to 
provide a health cover of up to Rs.5 lakh per family per year, for secondary and tertiary care 
hospitalization to over 10.74 crore entitled families as per SECC 2011 (approximately 50 crore 
beneficiaries), in a cashless and paperless manner through public and private empaneled 
hospitals.  
 
The National Health Authority (NHA) is committed to ensuring that beneficiaries are empowered 
and have access to accurate, complete and timely information in a transparent manner. With this 
spirit, 

##3.2. Text Refinement (Data Cleansing)

### 3.2.1. Level 1 Refinement
Remove unnecessary spaces, newlines, characters and numbering from `raw_text`<br>
**RegEx Patterns:** <br>
Substitute these patterns with required character.<br>
**A.** For Ordering Alphabets and Numbering Numerals and Roman: Substitute with '\<empty string\>'<br>
`pattern: ((\b(\d|\d\d)\.)|(\b(i|v|x)+\.)|(\b(I|V|X)+\.)|(\b([a-z]|[A-Z])\.($|\ )+))`
1. **`(\b(\d|\d\d)\.)`** `:- For Numbers like '1.','2.',...,'9.','10.','11.',...,'99.'`
2. **`((\b(i|v|x)+\.)|(\b(I|V|X)+\.))`**`:- For Roman Numbers like 'i.','ii.',...,'I.','II','III',...`
3. **`(r\b([a-z]|[A-Z])\.($|\ )+)`**`:- For Ordering Alphabets like 'a.','b.',...'z.','A.','B.',...'Z.'`<br>
**B.** For extra spaces, characters and newlines: Substitute with single space.<br>
`pattern:((\n|\s|!|#|%|\^|&|\*|\(|\))+)`<br>
1. **`(\s+)`**`:-For one or more spaces: Substitute with one space.`
2. **`(\n+)`**`:-For one or more new lines: Substitute with one space`
3. **`(!|#|%|\^|&|\*|\(|\))+`**`:-For unnecessary characters '!','$','%','^','&','*','(' and ')'.`
**C.** For multiple subsequent dots "....": Substitute with single dot "."
1. **`(\.+)`**:- For one or more dot/s "...": substitute with single dot ".".

In [ ]:
#Regular Expression Library
import re 

In [ ]:
#Level 1 Refinement Function for Raw Text
def refine_1(text): 
    #Ordering Alphabets, Numerals and Roman Numerals
    a= re.compile(r'(\b(\d|\d\d)\.)|(\b(i|v|x)+\.)|(\b(I|V|X)+\.)|(\b([a-z]|[A-Z])\.($|\ )+)') 
    b= re.compile(r'(\n|\s|!|#|%|\^|&|\*|\(|\))+')#New Lines, Spaces and Special characters
    c= re.compile(r'(\.+)') #Multiple dots (...)
    d= re.compile(r'(\d*\ Section\ \d\ *:\ *)') #Section
    e= re.compile(r'(\d*\ Chapter\ \d\ *:\ *)') #Chapter
    f= re.compile(r'(\d*\ Annexe\ \d\ *:\ *)') #Annexe
    g= re.compile(r'(^\ *\d*\ )') #Starting Numbers in a Sentence
    h= re.compile(r'•') # Marking Dot (•)
    i= re.compile(r'(^[aA-zZ]*\.*$)') #Single-Word Sentences
    text= i.sub('',(h.sub('',(g.sub('',(f.sub('',(e.sub('',(d.sub('',(c.sub('.',(b.sub(' ',(a.sub('',text)))))))))))))))))
    return(text)

In [ ]:
def text_to_sent_char(t):
    sent=pd.DataFrame()
    #print(sent)
    text=''
    for c in t:
        text=text+c
        if(c=='.'):
            sent= sent.append([text])
            #sent= sent.concat([text])
            text=''
    sent.rename(columns={0:'sentences'},inplace=True)
    sent= sent.reset_index(drop=True) #Resetting the Index
    return(sent)

In [ ]:
import spacy
nlp= spacy.load('en_core_web_sm')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
def text_to_sent_nlp(text):
    d=nlp(text)
    sentences=[]
    sent=''
    for tok in d:    
        if(tok.text=='.' or tok.text==':'):
            sent=sent+tok.text
            sentences.append(sent.strip())
            sent=''
        else:
            sent=sent+" "+tok.text

    sent_df=pd.DataFrame(sentences) #Converting the list into DataFrame for ease of visual analysis
    for i in range(0,len(sent_df)):
        for j in range(0,6):
            sent_df[0][i]=(refine_1(str(sent_df[0][i]))).strip() #Level 1 Refinement of Each Sentence done 5 times.
        if(sent_df[0][i]==''): #Checking for Empty Rows
            sent_df= sent_df.drop(i) #Removing Empty Rows
    sent_df= sent_df.reset_index(drop=True) #Resetting the Index
    sent_df.rename(columns={0:'sentences'},inplace=True) #Renaming the Column

    return(sent_df)

In [ ]:
#Level 1 Refinement
text_1= (refine_1(raw_text)).strip()

3.2.1.1. Getting OpenAI generated domain specific sentences from `ai_train_corpus.csv`

In [ ]:
#Getting AI Train Corpus
ai_text_df= pd.read_csv('/content/drive/MyDrive/Final (Completed)/ai_train_corpus2.csv')
text_2=''
#Level 1 Refinement
for i in range(0, len(ai_text_df['ent_pair_sentences'])):
    text_2= text_2 + refine_1(ai_text_df['ent_pair_sentences'][i])

In [ ]:
#Splitting the text into Sentences
sent_df_1= text_to_sent_nlp(text_1)
sent_df_2= text_to_sent_char(text_2)

<ipython-input-6-84230f734c5b>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent= sent.append([text])
<ipython-input-6-84230f734c5b>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent= sent.append([text])
<ipython-input-6-84230f734c5b>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent= sent.append([text])
<ipython-input-6-84230f734c5b>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent= sent.append([text])
<ipython-input-6-84230f734c5b>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent= sent.append([text])
<ipython-input-6-84230f73

In [ ]:
for i in range(0,len(sent_df_2['sentences'])):
    sent_df_2['sentences'][i]=(refine_1(sent_df_2['sentences'][i])).strip()
    if(sent_df_2['sentences'][i]==''):
        sent_df_2= sent_df_2.drop(i) #Removing Empty Rows
sent_df_2= sent_df_2.reset_index(drop=True) #Resetting the Index

In [ ]:
print('Lengths:\nsent_df_1:',len(sent_df_1),'\nsent_df_2:',len(sent_df_2))
sent_df= sent_df_1.append(sent_df_2)
print('sent_df:',len(sent_df))
sent_df= sent_df.reset_index(drop=True) #Resetting the Index

Lengths:
sent_df_1: 308 
sent_df_2: 1035
sent_df: 1343


<ipython-input-14-6b5720199e7e>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_df= sent_df_1.append(sent_df_2)


Now we have **`sent_df`**, which contains 819 level 1 refined sentences. These sentences are combination of Govt. Documents and Open AI Generated Sentences.

In [ ]:
sent_df

,sentences
0,Microsoft Word - Beneficiary Empowerment Guide...
1,PM - JAY is an ambitious government scheme whi...
2,The National Health Authority NHA is committed...
3,"With this spirit , NHA is sharing the Benefici..."
4,We sincerely hope that the SHAs and other stak...
...,...
1338,Ayushman Bharat PMJAY helps beneficiaries to a...
1339,Ayushman Bharat PMJAY empowers beneficiaries t...
1340,Ayushman Bharat PMJAY ensures that all empanel...
1341,Ayushman Bharat PMJAY provides a sense of secu...


#Level 2 Refinement
Prepare possible regular expressions for variations of domain specific entities that can be present in a text/sentence.<br>
`regex_pattern`: `A list of tuples (entity, reg_exp for variations of entity)`
<br>
Using this `regex_pattern` replace the variations with corresponding domain specific entity.<br>

**`Input: sent_df`**: Has Level 1 Refined text seperated into sentences and stored in a DataFrame.<br>
**`Output: text`**: This DataFrame will contain level 2 refined sentences which will go through the last Level 3 Refinement and generate `refined_text.csv`.

In [ ]:
#Regex Patterns for possible variations of unique entities
regex_pattern=[]
regex_pattern.append(('Am I Eligible App',(re.compile(r'(\'*[aA]+?m[\ -]*?[iI]+?[\ -]*?[eE]+?ligible[\ -]*?[aA]+?pp(lication)*s*([\ \']*?)*)'))))
regex_pattern.append(('Am I Eligible Website',(re.compile(r'(\'*[aA]+?m[\ -]*?[iI]+?[\ -]*?[eE]+?ligible[\ -]*?[wW]+?ebsites*([\ \']*?)*)'))))
regex_pattern.append(('BIS',(re.compile(r'((\bbis\b)|([bB]+?eneficiary\ *[iI]+?dentification\ *[sS]+?ystems*))'))))
regex_pattern.append(('Call-Centre',(re.compile(r'[cC]+?all[\ -]*?[cC]+?ent[re]?[er]?s*'))))
regex_pattern.append(('Claim',(re.compile(r'\b[cC]+?laims*'))))
regex_pattern.append(('Consultation',(re.compile(r'\b[cC]+?onsultations*'))))
regex_pattern.append(('Diagnostics',(re.compile(r'[dD]+?iagnostics*'))))
regex_pattern.append(('Doctor',(re.compile(r'\b[dD]+?octors*'))))
regex_pattern.append(('e-card',(re.compile(r'[eE]+?[-\ ]*?[cC]+?ards*'))))
regex_pattern.append(('EHCP',(re.compile(r'(\b[eE]+?[hH]+?[cC]+?[pP]+?\b)|[eE]+?mpanelled\ *[hH]+?ealth\ *[cC]+?are\ *[pP]+?roviders*'))))
regex_pattern.append(('EHCP Registration Desk',(re.compile(r'([eE]+?[hH]+?[cC]+?[pP]+?)+[-\ ]*?[rR]+?egistration[-\ ]*?([dD]+?esks*)+'))))
regex_pattern.append(('Empanelled Hospital',(re.compile(r'([eE]+?[hH]+?[cC]+?[pP]+?\ *)*([eE]+?mpanelled\ *)*[hH]+?ospitals*'))))
regex_pattern.append(('Feedback',(re.compile(r'[fF]+?eedbacks*'))))
regex_pattern.append(('General Doctor',(re.compile(r'[gG]+?eneral\ *[dD]+?octors*'))))
regex_pattern.append(('General Doctor or General Specialist',(re.compile(r'[gG]+?eneral\ *[dD]+?octors*((\ *|/)|(/|\ *))+([gG]+?eneral\ *)*[sS]+?pecialists*'))))
regex_pattern.append(('General Specialist',(re.compile(r'[gG]+?eneral\ *[sS]+?pecialists*'))))
regex_pattern.append(('Golden Record',(re.compile(r'[gG]+?olden[\ -]*?[rR]+ecords*'))))
regex_pattern.append(('Hospitalization',(re.compile(r'[hH]+?ispitali[zs]?ations*'))))
regex_pattern.append(('Insurance Company',(re.compile(r'(([iI]+?nsurers*)|([iI]+?nsurance\ *([cC]+?ompan(y|ies))*))'))))
regex_pattern.append(('ISA',(re.compile(r'(\bisa\b)|([iI]+?mplementations*\ *[sS]+?upport\ *[aA]+?genc(y|ies))'))))
regex_pattern.append(('MEDCO',(re.compile(r'([mM]+?edco)|([mM]+edical\ *[cC]+?o(-)*ordinators*)'))))
regex_pattern.append(('Medical Package',(re.compile(r'(([mM]+?edical\ *)*([pP]+?ackages*))'))))
regex_pattern.append(('Medical Report',(re.compile(r'(([mM]+?edical\ *)*([dD]+?iagnostics*\ *)*[rR]+eports*)|([cC]+?linical\ *[nN]+?otes*)|([dD]+?ischarge\ *[sS]+?ummar(y|ies))'))))
regex_pattern.append(('NHA',(re.compile(r'(\bnha\b)|([nN]+?ational\ *[hH]+?ealth\ *[aA]+?uthorit(y|ies))'))))
regex_pattern.append(('Patient',(re.compile(r'\bpatients*\b'))))
regex_pattern.append(('Picture',(re.compile(r'\bpictures*\b'))))
regex_pattern.append(('PMAM',(re.compile(r'(\b[pP]+?radhan\ *[mM]+?antri\ *[aA]+?rogya\ *[mM]+?itra\b)|\bpmam\b'))))
regex_pattern.append(('PMJAY',(re.compile(r'(\b[pP]+?radhan\ *[mM]+?antri\ *-*\ *[jJ]+?an\ *[aA]+?rogya\ *[yY]+?ojana\b)|(\b[pP]+?[mM]+?\ *-*\ *[jJ]+?[aA]+?[yY]+?\b)'))))
regex_pattern.append(('PMJAY',(re.compile(r'([aA]+?yushman [bB]+?harat\ [sS]+?cheme)'))))
regex_pattern.append(('PMJAY',(re.compile(r'([aA]+?yushman [bB]+?harat\ (PMJAY)*)'))))
regex_pattern.append(('PMJAY',(re.compile(r'((AYUSHMAN BHARAT\ )*PRADHAN\ *MANTRI\ *-*\ *JAN\ *AROGYA\ *YOJANA)'))))
regex_pattern.append(('PMJAY Beneficiary',(re.compile(r'(\bPMJAY\ *)*[bB]+?eneficiar(y|ies)'))))
regex_pattern.append(('PMJAY Kiosk',(re.compile(r'(\bPMJAY\ *)*[kK]+?iosks*'))))
regex_pattern.append(('Pre-Authorization Form',(re.compile(r'[pP]+?re\ *-\ *[aA]+?uthorization\ *[fF]orms*\b'))))
regex_pattern.append(('Pre-Authorization Request',(re.compile(r'[pP]+?re\ *-\ *[aA]+?uthorization\ *[rR]equests*\b'))))
regex_pattern.append(('SHA',(re.compile(r'(\b[sS]+?[hH]+?[aA]+?[sS]?\b)|([sS]+?tate\ *[hH]+?ealth\ *[aA]+?genc(y|ies))'))))
regex_pattern.append(('SHA or NHA',(re.compile(r'(SHA[sS]?\ */\ *NHA)|(NHA*\ */\ *SHA[sS]?)'))))
regex_pattern.append(('Silver Record',(re.compile(r'[sS]+?ilver[\ -]*?[rR]+ecords*'))))
regex_pattern.append(('TMS',(re.compile(r'(\b[tT]+?[mM]+?[sS]+?\b)|([tT]+?ransactions*\ *[mM]+?anagement\ *[sS]+?ystems*)'))))
regex_pattern.append(('Treating Doctor',(re.compile(r'[tT]+?reating\ *[dD]+?octors*'))))
regex_pattern.append(('Treating Doctor or Treating Specialist',(re.compile(r'[tT]+?reating\ *[dD]+?octors*((\ *|/)|(/|\ *))+([tT]+?reating\ *)*[sS]+?pecialists*'))))
regex_pattern.append(('Treating Specialist',(re.compile(r'[tT]+?reating\ *[sS]+?pecialists*'))))
regex_pattern.append(('Treatment',(re.compile(r'[tT]+?reatments*'))))
regex_pattern.append(('Treatment Procedure',(re.compile(r'[tT]+?reatment\ *[pP]+?rocedures*'))))
regex_pattern.append(('Treatment Report',(re.compile(r'[tT]+?reatment\ *[rR]+?eports*'))))

In [ ]:
def refine_2(sent):
    for p in regex_pattern:
        sent=p[1].sub(p[0],sent)
    return(sent)

In [ ]:
text=[]
for sent in sent_df['sentences']:
    text.append(refine_2(sent))
text=pd.DataFrame({'sentences':text})

In [ ]:
text

,sentences
0,Microsoft Word - PMJAY Beneficiary Empowerment...
1,PMJAY is an ambitious government scheme which ...
2,The NHA NHA is committed to ensuring that PMJA...
3,"With this spirit , NHA is sharing the PMJAY Be..."
4,We sincerely hope that the SHA and other stake...
...,...
1338,PMJAY helps PMJAY Beneficiary to access qualit...
1339,PMJAY empowers PMJAY Beneficiary to make infor...
1340,PMJAY ensures that all Empanelled Hospital adh...
1341,PMJAY provides a sense of security to PMJAY Be...


#Level 3 Refinement
Remove adjacent duplicate words from sentences of `text`

In [ ]:
def refine_3(sent):
    p= re.compile(r'\b(\w+)\s+\1\b')
    sent= p.sub(r'\1',sent)
    return(sent)

In [ ]:
refined_t=[]
for sent in text['sentences']:
    refined_t.append(refine_3(sent))
text=pd.DataFrame({'sentences':refined_t})
#text.rename(columns={0:'sentences'},inplace=True)    

In [ ]:
text

,sentences
0,Microsoft Word - PMJAY Beneficiary Empowerment...
1,PMJAY is an ambitious government scheme which ...
2,The NHA is committed to ensuring that PMJAY Be...
3,"With this spirit , NHA is sharing the PMJAY Be..."
4,We sincerely hope that the SHA and other stake...
...,...
1338,PMJAY helps PMJAY Beneficiary to access qualit...
1339,PMJAY empowers PMJAY Beneficiary to make infor...
1340,PMJAY ensures that all Empanelled Hospital adh...
1341,PMJAY provides a sense of security to PMJAY Be...


In [ ]:
text.to_csv('/content/drive/MyDrive/Final (Completed)/refined_text2.csv')

#End Of File